In [1]:
# Import 
import os
import time
import json
import datetime
import numpy as np
import pandas as pd

from IPython.display import display

from pathlib import Path
from tyssue import config
from tyssue import Sheet#, SheetGeometry as geom
from tyssue.geometry.sheet_geometry import ClosedSheetGeometry as geom
from tyssue.io import hdf5
from tyssue.io.hdf5 import load_datasets
from tyssue.dynamics import SheetModel as basemodel
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.solvers.sheet_vertex_solver import Solver as solver
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.draw.ipv_draw import sheet_view as ipv_draw 

from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.apoptosis_events import apoptosis

import matplotlib.pyplot as plt
%matplotlib inline

try:
    import seaborn as sns
except ImportError:
    print('Seaborn not found')
import ipyvolume as ipv
ipv.clear()

In [3]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/2019-05-23')
dirname = SIM_DIR/'1.04_constriction_30_radialtension0/'
from tyssue.draw import sheet_view

import ipyvolume.pylab as p3
hfs = [f  for f in os.listdir(dirname)
       if f.endswith('hf5')]
hfs.sort(key=lambda x: os.path.getctime(os.path.join(dirname, x)))

current = 'invagination_0030.hf5'
ipv.clear()
ipv.style.use('dark')


dsets = hdf5.load_datasets(os.path.join(dirname, current),
                               data_names=['vert', 'edge', 'face', 'cell'])

specs = config.geometry.cylindrical_sheet()
sheet = Sheet('ellipse', dsets, specs)
sheet.face_df['test']=0
index =sheet.face_df[(sheet.face_df.z>6) & (sheet.face_df.z<15) 
              & (sheet.face_df.y>25) & (sheet.face_df.y<35) 
                    & (sheet.face_df.x>0)].index
sheet.face_df.loc[index, 'test']=1

#sheet = sheet.extract_bounding_box(z_boundary=[-200,50])
def draw_specs(sheet):
    specs = {
        'edge': {
            'visible': True,
            'color': sheet.vert_df.y,
            #'colormap': 'Reds'
        },
        'face': {
            'visible': True,
            'color': sheet.face_df.test,
            'colormap': 'Blues'
        },
        'vert':{
            'visible':True,
            's':100,
            'alpha': 0.3,
            'zorder': 2,
            'color': sheet.vert_df.radial_tension
            
        }
        
    }
    return specs

ipv.clear()
ipv_fig, meshes = sheet_view(sheet, coords=list('zxy'),
                             mode='3D', **draw_specs(sheet))
ipv_fig

Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array([[0.123463, 0.581687, 0.547445],
       [0.15627 , 0.489624, 0.557936],
       [0.126453, 0.570633, 0.549841],
       ...,
       [0.720391, 0.87035 , 0.162603],
       [0.720391, 0.87035 , 0.162603],
       [0.699415, 0.867117, 0.175971]]), lines=array([[   0,  258],
       [3049, 3046],
       [1971, 3049],
       ...,
       [1583, 1582],
       [ 685, 1157],
       [3737, 3734]], dtype=uint32), texture=None, x=array([-113.39480893,  -41.72852124, -117.00508765, ...,  -38.56851822,
        -35.05489284,  -36.77418204]), y=array([-16.93614014,  37.67622766,   6.96536096, ..., -24.54013608,
       -24.63742598, -25.3796001 ]), z=array([ 1.23078331, -6.38625824,  0.34250412, ..., 30.402936  ,
       30.35665219, 29.74462016])), Mesh(color=array([[0.96862745, 0.98431373, 1.        ],
       [0.96862745, 0.98431373, 1.        ],
       [0.96862745, 0.98431373, 1.        ],
       ...,
       [0.96862745, 0.98431373, 1.        ],
       [0.96862745, 0.98431373, 1.        ],
       [0.96862745, 0.98431373, 1.        ]]), texture=None, triangles=array([[    0,  2816, 19700],
       [ 1495,  2817, 19701],
       [ 1495,  2818, 19702],
       ...,
       [  720, 19697, 36581],
       [  718, 19698, 36582],
       [  719, 19699, 36583]], dtype=uint32), x=array([-111.89633846, -114.06845727, -113.92742561, ...,  -27.96808378,
        -15.8054846 ,  -10.54442521]), y=array([-19.15867394, -15.53726417, -15.41887176, ...,  13.46270264,
        17.89729986,  12.09313585]), z=array([  0.83579376,  -0.91707226,   3.28323208, ..., -36.14588169,
       -33.57910944, -35.81822482]))], style={'axes': {'visible': False, 'label': {'color': 'white'}, 'ticklabel': {'color': 'white'}, 'color': 'white'}, 'box': {'visible': False}, 'background-color': 'black'}, tf=None, width=400, xlim=[-129.39586216729785, 129.41339856969765], ylim=[-129.39586216729785, 129.41339856969765], zlim=[-129.39586216729785, 129.41339856969765])

In [10]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/2019-05-26_clone_eloigne_propagation')
dirname = SIM_DIR/'1.08_constriction_50_radialtension0/'

hfs = [f  for f in os.listdir(dirname) if f.endswith('hf5')]
hfs.sort()
from tyssue.draw.ipv_draw import edge_mesh, face_mesh

def get_meshes(dirname, hdf_file):
    dsets = hdf5.load_datasets(os.path.join(dirname, hdf_file),
                               data_names=['vert', 'edge', 'face'])
        
    sheet = Sheet('ellipse', dsets)
    faces = face_mesh(sheet, coords=['x', 'y', 'z'],
                      color=sheet.face_df['area'],
                      color_range=(0, 30))
    
    edges = edge_mesh(sheet, coords=['x', 'y', 'z'],
                      color=sheet.vert_df.y)
    return faces, edges

from ipywidgets import interact

def browse_sheets(dirname, hfs):
    n = len(hfs)
    ipv.clear()
    fig = ipv.gcf()
    meshes = get_meshes(dirname, hfs[0])
    fig.meshes = fig.meshes + list(meshes)
    
    def view_sheet(i):
        fig.animation = 0
        sheet.face_df['test']=0
        faces, edges = get_meshes(dirname, hfs[i])
        print()
        fig.meshes[0].x = faces.x
        fig.meshes[0].y = faces.y
        fig.meshes[0].z = faces.z
        fig.meshes[0].color = faces.color
        fig.meshes[0].triangles = faces.triangles

        fig.meshes[1].x = edges.x
        fig.meshes[1].y = edges.y
        fig.meshes[1].z = edges.z
        fig.meshes[1].color = edges.color
        fig.meshes[1].lines = edges.lines

        lim_inf = sheet.vert_df[sheet.coords].min().min()
        lim_sup = sheet.vert_df[sheet.coords].max().max()
        ipv.xyzlim(lim_inf, lim_sup)
    ipv.show()
    interact(view_sheet, i=(0, n-1))

In [11]:
browse_sheets(dirname, hfs)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array([[0.126326, 0.644107, 0.525311],
       [0.12138 , 0.629492, 0.531973],
       [0.128087, 0.647749, 0.523491],
       ...,
       [0.214   , 0.722114, 0.469588],
       [0.214   , 0.722114, 0.469588],
       [0.214   , 0.722114, 0.469588]]), texture=None, triangles=array([[    0,  2816, 19700],
       [ 1495,  2817, 19701],
       [ 1495,  2818, 19702],
       ...,
       [  720, 19697, 36581],
       [  718, 19698, 36582],
       [  719, 19699, 36583]], dtype=uint32), x=array([-19.08217641, -15.51566974, -15.39688422, ...,  14.64801651,
        19.95982303,  14.74428955]), y=array([  0.79794613,  -0.94109734,   3.22120845, ..., -36.02356581,
       -33.61653647, -36.26922054]), z=array([-111.77870289, -113.96757793, -113.83048896, ...,  -28.40628845,
        -16.33946974,  -11.17641634])), Mesh(color=array([[0.124395, 0.578002, 0.548287],
       [0.15627 , 0.489624, 0.557936],
       [0.126453, 0.570633, 0.549841],
       ...,
       [0.709898, 0.868751, 0.169257],
       [0.720391, 0.87035 , 0.162603],
       [0.688944, 0.865448, 0.182725]]), lines=array([[   0,  258],
       [3049, 3046],
       [1971, 3049],
       ...,
       [1583, 1582],
       [ 685, 1157],
       [3737, 3734]], dtype=uint32), texture=None, x=array([-16.89476652,  37.93108382,   6.76445168, ..., -23.68689837,
       -23.75945067, -24.50658949]), y=array([ 1.18713572, -6.22639715,  0.30513005, ..., 30.49260506,
       30.5824578 , 29.91673806]), z=array([-113.287997  ,  -42.25387345, -117.04607128, ...,  -38.77031417,
        -35.30613886,  -37.03721817]))], style={'axes': {'visible': False, 'label': {'color': 'white'}, 'ticklabel': {'color': 'white'}, 'color': 'white'}, 'box': {'visible': False}, 'background-color': 'black'}, tf=None, width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]),))

interactive(children=(IntSlider(value=99, description='i', max=199), Output()), _dom_classes=('widget-interact',))